# Imports

using Efficientnet

In [14]:
!pip install -q efficientnet wandb

In [15]:
from glob import glob
import math, random, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow_addons.metrics import F1Score

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import KFold

import efficientnet.tfkeras as efn

from kaggle_datasets import KaggleDatasets

import wandb
from wandb.integration.keras import WandbCallback

In [16]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret('wandb_key')
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your \
           W&B access token. Use the Label name as WANDB. \nGet your W&B access \
           token from here: https://wandb.ai/authorize')

In [17]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Configurations

Setting image size to 224x224 and no of epochs to 20.

In [18]:
AUTO = tf.data.experimental.AUTOTUNE

IMAGE_SIZE = 224
EPOCHS = 20

SEED = 6969
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
CLASSES = [0,1]

# Mixed Precision - XLA

In [19]:
MIXED_PRECISION = True
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

# Data Augmentation

* zooming and shifting the image 
* random flip left and right
* random brightness
* random contrast
* random saturation

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

In [ ]:
def data_augment(image, label):

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1, seed=69)
    image = tf.image.random_contrast(image, 0.8, 1.0)
    image = tf.image.random_saturation(image, 1, 2)

    return image, label

# Loading dataset

In [25]:
train_path_0 = '../input/car-damage-detection/data1a/training/00-damage/*'
train_path_1 = '../input/car-damage-detection/data1a/training/01-whole/*'

train_0 = sorted(glob(train_path_0))
train_1 = sorted(glob(train_path_1))

print(len(train_0), len(train_1))

val_path_0 = '../input/car-damage-detection/data1a/validation/00-damage/*'
val_path_1 = '../input/car-damage-detection/data1a/validation/01-whole/*'

valid_0 = sorted(glob(val_path_0))
valid_1 = sorted(glob(val_path_1))

print(len(valid_0), len(valid_1))

map_train = {}
map_val = {}
for i in range(len(train_0)):
    map_train[train_0[i]] = 0

for i in range(len(train_1)):
    map_train[train_1[i]] = 1
    
for i in range(len(valid_0)):
    map_val[valid_0[i]] = 0

for i in range(len(valid_1)):
    map_val[valid_1[i]] = 1

# Utitlity function to create tf.dataset

Using tf.dataset to generate dataset

In [26]:
def decode_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    return image

def get_labels_from_filenames(filenames, train):
    if train:
        labels = []
        for name in filenames:

            labels.append(map_train[name])
        labels = tf.keras.utils.to_categorical(
        labels, num_classes=len(CLASSES), dtype='float32'
    )
        
    else:
        labels = []
        for name in filenames:

            labels.append(map_val[name])
        labels = tf.keras.utils.to_categorical(
        labels, num_classes=len(CLASSES), dtype='float32'
    )
    return labels

def load_dataset(path_0, path_1, train):
    
    filenames = tf.io.gfile.glob(path_0) + tf.io.gfile.glob(path_1)
    random.shuffle(filenames)
    rnd_labels = get_labels_from_filenames(filenames, train)
    
    DATASET_SIZE = len(filenames)
    print(len(filenames))

    filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
    images_ds = filenames_ds.map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    labels_ds = tf.data.Dataset.from_tensor_slices(rnd_labels)
    dataset = tf.data.Dataset.zip((images_ds, labels_ds))
        
    return dataset, DATASET_SIZE

def get_training_and_validation_dataset():
    train_ds, TRAIN_DS_SIZE = load_dataset(train_path_0, train_path_1, train = True)

    train_ds = train_ds.map(data_augment, num_parallel_calls=AUTO)
    train_ds = train_ds.map(transform, num_parallel_calls=AUTO)
    
    train_ds = train_ds.repeat()
    train_ds = train_ds.batch(BATCH_SIZE)
    train_ds = train_ds.prefetch(AUTO)
    
    val_ds, VAL_DS_SIZE = load_dataset(val_path_0, val_path_1, train = False)
    
    val_ds = val_ds.batch(BATCH_SIZE)
    val_ds = val_ds.cache()
    val_ds = val_ds.prefetch(AUTO)
    
    return train_ds, TRAIN_DS_SIZE, val_ds, VAL_DS_SIZE

In [27]:
print(len(train_0)+len(train_1), len(map_train))
print(len(valid_0)+len(valid_1), len(map_val))

# Visualizing the augmentations

In [29]:
for i in range(5):
    row = 3; col = 4;
    train_ds, NUM_TRAINING_IMAGES, val_ds, NUM_VALIDATION_IMAGES = get_training_and_validation_dataset()
    all_elements = train_ds.unbatch()
    one_element = tf.data.Dataset.from_tensors(next(iter(all_elements)) )
    one_element_ = tf.data.Dataset.from_tensors(next(iter(all_elements)) )
    augmented_element = one_element_.repeat().map(data_augment).batch(row*col)

    for (img,label) in augmented_element:
        plt.figure(figsize=(15,int(15*row/col)))
        for j in range(row*col):
            plt.subplot(row,col,j+1)
            plt.axis('off')
            plt.imshow(img[j,])
        plt.show()
        break

# Train and Infer

Learning rate schedule as using an high LR would break the pre-trained weights so, ramp up is utilized to fine-tune a pre-trained model.

In [30]:
LR_START = 0.00001/2.0
LR_MAX = 0.00001 * strategy.num_replicas_in_sync
LR_MIN = 0.00001/2.0
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

# Training the model

* optimizer - ADAM
* loss - Binary Crossentropy (as it is binary classification)
* metrics - accuracy

In [32]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

from tensorflow.keras.applications import DenseNet201

def get_model():
    with strategy.scope():
        rnet = efn.EfficientNetB7(
            input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
            weights='noisy-student',
            include_top=False
        )
        rnet.trainable = True
        model = tf.keras.Sequential([
            rnet,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(len(CLASSES), activation='softmax',dtype='float32')
        ])
    model.compile(
        optimizer='adam',
        loss = 'BinaryCrossentropy',
        metrics=['accuracy']
    )
    return model

def train_and_validate():
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
        
    train_ds, NUM_TRAINING_IMAGES, val_ds, NUM_VALIDATION_IMAGES = get_training_and_validation_dataset()

    model = get_model()
    wandb.init(project='Task 2.1 car-scan', job_type='train', reinit=True)
    log_callback = WandbCallback()
    history = model.fit(
        train_ds, 
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = EPOCHS,
        callbacks = [lr_callback, log_callback, early_stopping],
        validation_data = val_ds,
        verbose=1
    )
    wandb.finish()
    return history, model

def train_():

    history, model = train_and_validate()
    return history, model
    
# run train and predict
history, model = train_()

In [ ]:
model.save('./model')

In [ ]:
import keras
model = keras.models.load_model('./model')

In [ ]:
model.summary()

In [ ]:
!mkdir model

In [ ]:
import shutil
shutil.make_archive('./model', 'zip', './model')